In [ ]:
from lxml import etree

# Read the Sanskrit text
with open("sanskrit.txt", "r", encoding="utf-8") as file:
    content = file.readlines()

# Create a TEI XML structure
root = etree.Element("TEI", xmlns="http://www.tei-c.org/ns/1.0")
tei_header = etree.SubElement(root, "teiHeader")
file_desc = etree.SubElement(tei_header, "fileDesc")
title_stmt = etree.SubElement(file_desc, "titleStmt")
title = etree.SubElement(title_stmt, "title").text = "Sanskrit Book Title"
author = etree.SubElement(title_stmt, "author").text = "Author Name"

# Add body content
text = etree.SubElement(root, "text")
body = etree.SubElement(text, "body")

# Divide text into chapters (example: every 10 li`nes is a chapter)
chapter = etree.SubElement(body, "div", type="chapter", n="1")
for line in content:
    etree.SubElement(chapter, "p").text = line.strip()

# Write to TEI XML
tree = etree.ElementTree(root)
tree.write("sanskrit.xml", pretty_print=True, xml_declaration=True, encoding="utf-8")


In [4]:
from lxml import etree
import sqlite3

# Parse the TEI XML
tree = etree.parse("sanskrit.xml")
root = tree.getroot()

# Define TEI namespace
ns = {"tei": "http://www.tei-c.org/ns/1.0"}

# Extract Metadata with Namespace
title_nodes = root.xpath("//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title", namespaces=ns)
author_nodes = root.xpath("//tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:author", namespaces=ns)

# Check and Extract Title and Author
title = title_nodes[0].text if title_nodes else "Unknown Title"
author = author_nodes[0].text if author_nodes else "Unknown Author"

print(f"Title: {title}")
print(f"Author: {author}")


Title: Sanskrit Book Title
Author: Author Name


In [4]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("sanskrit_1.db")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Metadata (
    book_id INTEGER PRIMARY KEY,
    title TEXT,
    author TEXT,
    year INTEGER,
    pdf BLOB,
    text TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Content (
    content_id INTEGER PRIMARY KEY,
    book_id INTEGER,
    chapter INTEGER,
    text TEXT,
    FOREIGN KEY(book_id) REFERENCES Metadata(book_id)
)
""")

# Save and Close
conn.commit()
conn.close()

print("Data successfully inserted into the database.")


Data successfully inserted into the database.


In [5]:
import sqlite3

# Function to execute database queries
def query_database(query, params=()):
    conn = sqlite3.connect("sanskrit.db")
    cursor = conn.cursor()
    cursor.execute(query, params)
    results = cursor.fetchall()
    conn.close()
    return results

# Query the database
book_id = 1  # Replace with the desired book ID
metadata = query_database("SELECT title, author, year, pdf FROM Metadata WHERE book_id = ?", (book_id,))
print(metadata)  # Debugging step to check the contents


[('Sanskrit Book Title', 'Author Name', 2024, None)]


In [2]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("sanskrit.db")  # Opens or creates a database file
cursor = conn.cursor()

try:
    # Delete all rows from the Metadata table
    cursor.execute("DELETE FROM Metadata;")
    
    # Commit changes to the database
    conn.commit()
    print("All records from 'Metadata' table have been deleted successfully.")

except sqlite3.Error as e:
    # Rollback in case of any error
    conn.rollback()
    print(f"An error occurred: {e}")

finally:
    # Close the database connection
    cursor.close()
    conn.close()


An error occurred: database is locked


In [5]:
import time
import sqlite3

for attempt in range(5):  # Retry 5 times
    try:
        conn = sqlite3.connect("sanskrit.db")
        conn.execute("PRAGMA journal_mode=WAL;")
        print("WAL mode set successfully.")
        conn.close()
        break
    except sqlite3.OperationalError as e:
        if "database is locked" in str(e):
            print("Database is locked. Retrying...")
            time.sleep(1)
        else:
            raise


Database is locked. Retrying...
Database is locked. Retrying...
Database is locked. Retrying...
Database is locked. Retrying...
Database is locked. Retrying...


In [1]:
import sqlite3

def clear_table(table_name):
    """
    Clears all data from the specified table in the database.
    
    Parameters:
        table_name (str): The name of the table to clear.
    """
    try:
        conn = sqlite3.connect("sanskrit.db")  # Replace with your database name
        cursor = conn.cursor()
        cursor.execute(f"DELETE FROM {table_name}")
        conn.commit()
        print(f"All data from table '{table_name}' has been cleared.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

# Example usage
clear_table("Metadata")


All data from table 'Metadata' has been cleared.


In [2]:
import sqlite3

def clear_table(table_name):
    """
    Clears all data from the specified table in the database.
    
    Parameters:
        table_name (str): The name of the table to clear.
    """
    try:
        conn = sqlite3.connect("sanskrit.db")  # Replace with your database name
        cursor = conn.cursor()
        cursor.execute(f"DELETE FROM {table_name}")
        conn.commit()
        print(f"All data from table '{table_name}' has been cleared.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

# Example usage
clear_table("Content")


All data from table 'Content' has been cleared.


In [3]:
import sqlite3

def vacuum_database(db_path):
    """
    Shrinks the SQLite database file size by reclaiming unused space.
    
    Parameters:
        db_path (str): The path to the SQLite database file.
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("VACUUM")
        conn.commit()
        print(f"Database file '{db_path}' has been vacuumed and reduced in size.")
    except sqlite3.Error as e:
        print(f"An error occurred during vacuuming: {e}")
    finally:
        conn.close()

# Example usage
vacuum_database("sanskrit.db")


Database file 'sanskrit.db' has been vacuumed and reduced in size.
